In [ ]:
import os
from argparse import ArgumentParser
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
from im import current_dynamics
from constants import sigmax, sigmay, sigmaz
from cli_utils import _hdf2im, _hdf2trained_im, _cptp_rank_reduction

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 20,
})

script_dir = os.path.dirname(os.path.abspath(''))

# ----------------------------------------------------------------------------------- #

plotting_configs = [

    # first plot
    {
        "left_ims":  # left im's stumps
            [
                "J_0.1_do/2023-09-01_11:07:27+0200",
                "small_J_0.1_do/2023-09-04_22:37:54+0200",
                "ultra_small_J_0.1_do/2023-09-05_10:14:07+0200",
            ],
        "right_ims":  # right im's stumps
            [
                "J_0.1_up/2023-08-31_22:30:38+0200",
                "small_J_0.1_up/2023-09-04_22:36:20+0200",
                "ultra_small_J_0.1_up/2023-09-05_10:12:41+0200",
            ],
        "legend":  # plot's legend
            [
                r"$N=5\cdot 10^6$",
                r"$N=1\cdot 10^6$",
                r"$N=0.2\cdot 10^6$",
                r"${\rm ab \ initio}$",
            ],
        "styles": ["b--", "g-.","g:"],  # curves styles
        "xx": 0.1,  # XX interaction amplitude between impurity's spins
        "yy": 0.1,  # YY interaction amplitude between impurity's spins
        "zz": 0.1,  # ZZ interaction amplitude between impurity's spins
        "xl": 0.0,  # X field applied to the left spin
        "xr": 0.0,  # X field applied to the right spin
        "yl": 0.0,  # Y field applied to the left spin
        "yr": 0.0,  # Y field applied to the right spin
        "zl": 0.0,  # Z field applied to the left spin
        "zr": 0.0,  # Z field applied to the right spin
        "dens":  # initial impurity's density matrix
            np.array([
                0., 0., 0., 0.,
                0., 0., 0., 0.,
                0., 0., 1., 0.,
                0., 0., 0., 0., 
            ]),
    },

    # second plot
    {
        "left_ims":  # left im's stumps
            [
                "J_0.2_do/2023-09-01_19:53:01+0200",
            ],
        "right_ims":  # right im's stumps
            [
                "J_0.2_up/2023-09-02_19:19:37+0200",
            ],
        "legend":  # plot's legend
            [
                r"$N=5\cdot 10^6$",
                r"${\rm ab \ initio}$",
            ],
        "styles": ["b--"],  # curves styles
        "xx": 0.2,  # XX interaction amplitude between impurity's spins
        "yy": 0.2,  # YY interaction amplitude between impurity's spins
        "zz": 0.2,  # ZZ interaction amplitude between impurity's spins
        "xl": 0.0,  # X filed applied to the left spin
        "xr": 0.0,  # X filed applied to the right spin
        "yl": 0.0,  # Y filed applied to the left spin
        "yr": 0.0,  # Y filed applied to the right spin
        "zl": 0.0,  # Z filed applied to the left spin
        "zr": 0.0,  # Z filed applied to the right spin
        "dens":  # initial impurity's density matrix
            np.array([
                0., 0., 0., 0.,
                0., 0., 0., 0.,
                0., 0., 1., 0.,
                0., 0., 0., 0., 
            ]),
    },
]

# output *.pdf path
output_path = f"{script_dir}/experiments/output/physical_im/{plotting_configs[0]['left_ims'][0]}/xxx_current_dynamics.pdf"
columns_number = 2  # number of plot's columns
rows_number = 1  # number of plot's rows

# ----------------------------------------------------------------------------------- #

fig = plt.figure(figsize=(columns_number * 6.4 / 1.5, rows_number * 4.8 / 1.5))
for k, config in enumerate(plotting_configs):
    left_ims_path = [f"{script_dir}/experiments/output/physical_im/{left_im}" for left_im in config["left_ims"]]
    right_ims_path = [f"{script_dir}/experiments/output/physical_im/{right_im}" for right_im in config["right_ims"]]

    im_exact1 = _hdf2im(left_ims_path[0])
    im_exact2 = _hdf2im(right_ims_path[0])
    im_trained1 = [list(reversed(_hdf2trained_im(path))) for path in left_ims_path]
    im_trained2 = [list(reversed(_hdf2trained_im(path))) for path in right_ims_path]

    h = config["xx"] * np.kron(sigmax, sigmax) +\
        config["yy"] * np.kron(sigmay, sigmay) +\
        config["zz"] * np.kron(sigmaz, sigmaz) +\
        config["xl"] * np.kron(sigmax, np.eye(2)) +\
        config["xr"] * np.kron(np.eye(2), sigmax) +\
        config["yl"] * np.kron(sigmay, np.eye(2)) +\
        config["yr"] * np.kron(np.eye(2), sigmay) +\
        config["zl"] * np.kron(sigmaz, np.eye(2)) +\
        config["zr"] * np.kron(np.eye(2), sigmaz)
    u = expm(-1j * h).reshape((2, 2, 2, 2))
    phi = np.tensordot(u, u.conj(), axes=0)
    phi = phi.transpose((0, 4, 1, 5, 2, 6, 3, 7)).reshape((4, 4, 4, 4))
    current_gen = current_dynamics(im_exact1, im_exact2, len(im_exact1) * [phi], config["dens"])
    current_trained = [current_dynamics(i1, i2, len(i1) * [phi], config["dens"]) for i1, i2 in zip(im_trained1, im_trained2)]
    plt.subplot(rows_number, columns_number, k + 1)
    plt.plot(np.abs(current_gen), 'r-', label=config["legend"][-1])
    for c, s, l in zip(reversed(current_trained), reversed(config["styles"]), reversed(config["legend"][:-1])):
        plt.plot(np.abs(np.array(c)), s, label=l)
    current_gen = np.array(current_gen)
    plt.yscale("log")
    plt.xscale("log")
    plt.legend(frameon=False, labelspacing=0.02, handlelength=1.5, fontsize="15")
fig.subplots_adjust(hspace=0.3)
fig.text(0.5, -0.05, r"${\rm Time}$", ha='center')
fig.text(0.04, 0.5, r"$|I|$", va='center', rotation='vertical')
fig.show()

In [ ]:
fig.savefig(output_path, bbox_inches = 'tight')